In [ ]:
import torch
import networkx as nx
import numpy as np

# We need to set the following parameters:

'''
- gamma (edge penalty),
- gamma_c (clique param),
- alpha (learning rate),
- beta (momentum),
- T (number of interations to converge per initialization), and
- eta (explration parameter)
'''

'\n- gamma (edge penalty), \n- gamma_c (clique param), \n- alpha (learning rate), \n- beta (momentum), \n- T (number of interations to converge per initialization), and \n- eta (explration parameter)\n'

# This Code does a quick tuning of the hyper-parameters of pCQO-MIS to ensure feaisble solutions.

Note that to obtain the best results from pCQO, we need more tuning.

## Steps:
- Select $\gamma'$. Below, we use $\gamma' = 1$.
- From our Theorem, we have $\gamma > 1 +\gamma'\Delta(G')$ where $\Delta(G')$ is the maximum degree of the compliment graph $G'$. As such, here, we set $\gamma = 2+\Delta(G')$.
- Select $\beta$. Below, we use $\beta = 0.9$.
- Iterate over a set of step sizes. For example, $\alpha \in \{0.0001, 0.0005, 0.0001, 0.005, 0.01, 0.05\}$.
- At each $\alpha$, we use multiple initilizations. We use a while loop to record the number of steps needed for convergence in under a limit or $T$. Below, we use $T=500$.
- Log the best MIS, $T$, and number of times we get a feasible solution over the number of initializations.
- Then, select the smallest $T$ based on the largest MIS and the ratio of obtaining solutions.
- Set $\eta \geq 2.5$ for the variance of the Gaussian distribution used to obtain the initializations of the next batch. This is not needed here.


In what follow, we provide an example. This was tested on random ER graphs with $n$ up to 1000, and $p\in [0.01,0.9]$ where $p$ is the probability of edge creation. This $p$ is also an indication of the graph density. $d$-regular graphs (where $d$ is the degree) were also tested.

An example of the output of this notebook is:

++++++++++++ FINAL ++++++++++++

gamma   =  [149]

gamma_c =  [1]

alpha   =  [0.05]

beta    =  [0.8]

T       =  [np.int64(73)]

++++++++++++++++++++++++++++++

In our pcqmis.cpp, these correspond to:

++++++++++++ FINAL ++++++++++++

gamma   =  GAMMA

gamma_c =  GAMMA_PRIME

alpha   =  LEARNING_RATE

beta    =  MOMENTUM

T       =  NUM_ITERATIONS_PER_BATCH

++++++++++++++++++++++++++++++

In [ ]:
# Graph


######## ER ###########
n = 300
p = 0.6
G = nx.gnp_random_graph(n, p, seed=0)


# ######## d reg ###########
# n = 1000
# d = 10
# G = nx.random_regular_graph(d, n)


# compliment graph:
complement_G = nx.complement(G)

num_components = nx.number_connected_components(G)

print("Number of connected components:", num_components)

if num_components == 1:
    print("Graph has one connected component.")
else:
    print("Graph has more than one connected component.")

### Obtain the A_G matrix

adjacency_matrix = nx.adjacency_matrix(G)
adjacency_matrix_dense = adjacency_matrix.todense()
adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

### Obtain the A_G_hat matrix

adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)

Number of connected components: 1
Graph has one connected component.


In [ ]:
### Efficient MIS Checking 2: Directly check if a binarized X is a maximal IS without checking whether it is an IS first...

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
  n = X_torch.shape[0]
  # binarized X
  X_torch_binarized = X_torch.bool().float()
  # if for some gradient update, we are still at the boundary, then we have maximal IS
  X_torch_binarized_update = X_torch_binarized - 0.1*(-torch.ones(n) + (n*adjacency_matrix_tensor)@X_torch_binarized)
  # Projection to [0,1]
  X_torch_binarized_update[X_torch_binarized_update>=1] =1
  X_torch_binarized_update[X_torch_binarized_update<=0] =0
  if torch.equal(X_torch_binarized, X_torch_binarized_update):
    # we have a maximal IS:
    MIS = torch.nonzero(X_torch_binarized).squeeze()
    # Exit the function with True
    return True, MIS
  # If not a maximal IS, Exit the function with False
  return False, None

In [ ]:
### Here, we set gamma, gamma', and beta.

gamma_c = 1
degrees_c = dict(complement_G.degree())
max_degree = max(degrees_c.values())
gamma = 2 + max_degree
beta = 0.8

In [ ]:
### Define the alpha set to try

alpha_set = [0.000001, 0.000005,0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

In [ ]:
### optimization loop using MGD to obtain T


def opt_loop_MGD(seed, adjacency_matrix_tensor, adjacency_matrix_tensor_comp,gamma, gamma_c, alpha, beta):
  n = adjacency_matrix_tensor.shape[0]
  torch.manual_seed(seed)
  input_tensor = torch.rand(n)
  input_velocity = torch.zeros_like(input_tensor)

  Checker,MIS = MIS_checker_efficient_3(input_tensor, adjacency_matrix_tensor, adjacency_matrix_tensor_comp)
  iteration_T = 0
  while Checker == False:

      iteration_T += 1

      # Compute the gradient
      gradient = -torch.ones(n) + (gamma*adjacency_matrix_tensor - gamma_c*adjacency_matrix_tensor_comp)@input_tensor

      input_velocity = beta*input_velocity + alpha*gradient

      input_tensor = torch.clamp(input_tensor - input_velocity, 0, 1)

      Checker,MIS = MIS_checker_efficient_3(input_tensor, adjacency_matrix_tensor, adjacency_matrix_tensor_comp)

      if Checker:
        print(iteration_T, Checker, len(MIS))
        break

      if iteration_T > 500:
        iteration_T = 0
        MIS = [0]
        break

  return [iteration_T, Checker, len(MIS)]



In [ ]:
###

number_of_initializations = 5

T_and_MIS_per_alpha = {}

for alpha in alpha_set:
  print("+++++++ We are at alpha = ", [alpha])
  MGD_MIS=[]
  MGD_iter=[]
  T_and_MIS_per_alpha[alpha] = []
  ctr = 0
  for seed in range(number_of_initializations):
    print("We are at seed = ", [seed])

    temp = opt_loop_MGD(seed, adjacency_matrix_tensor, adjacency_matrix_tensor_comp, gamma, gamma_c, alpha, beta)
    if temp[2] > 1:
      ctr += 1
    MGD_MIS.append(temp[2])
    MGD_iter.append(temp[0])

  # Here, we log the avg T over seeds, max T over seeds, avg MIS over seeds, mas MIS over seeds, and percentage of obtaining solutions (number of times we got a sol over the number of inits we solved)
  T_and_MIS_per_alpha[alpha] = [np.mean(MGD_iter), np.max(MGD_iter), np.mean(MGD_MIS), np.max(MGD_MIS), ctr/number_of_initializations]
  print('Mean MIS size', np.mean(MGD_MIS))
  print('Mean T', np.mean(MGD_iter))

#print('T to select', np.max(MGD_iter))



+++++++ We are at alpha =  [1e-06]
We are at seed =  [0]
We are at seed =  [1]
We are at seed =  [2]
We are at seed =  [3]
We are at seed =  [4]
Mean MIS size 1.0
Mean T 0.0
+++++++ We are at alpha =  [5e-06]
We are at seed =  [0]
We are at seed =  [1]
We are at seed =  [2]
We are at seed =  [3]
We are at seed =  [4]
Mean MIS size 1.0
Mean T 0.0
+++++++ We are at alpha =  [1e-05]
We are at seed =  [0]
469 True 8
We are at seed =  [1]
469 True 8
We are at seed =  [2]
488 True 8
We are at seed =  [3]
391 True 10
We are at seed =  [4]
Mean MIS size 7.0
Mean T 363.4
+++++++ We are at alpha =  [5e-05]
We are at seed =  [0]
177 True 7
We are at seed =  [1]
251 True 7
We are at seed =  [2]
188 True 8
We are at seed =  [3]
244 True 6
We are at seed =  [4]
154 True 7
Mean MIS size 7.0
Mean T 202.8
+++++++ We are at alpha =  [0.0001]
We are at seed =  [0]
163 True 6
We are at seed =  [1]
110 True 7
We are at seed =  [2]
145 True 6
We are at seed =  [3]
94 True 8
We are at seed =  [4]
97 True 7
M

In [ ]:
# final evaluation:

# We need to check the number of times we obtained a solution with the limit of T out of the number of seeds

# The following dictionary log:
"""
avg T over seeds,
max T over seeds,
avg MIS over seeds,
max MIS over seeds,
and percentage of obtaining solutions (number of times we got a sol over the number of inits we solved)

Criteria:
- Select the smallest T based on the largest MIS and the ratio of obtaining solutions.

"""

# First: filter out the ones where the ratio is not 1 (i.e., the selection of the parameters returned a solution for all the initilizations):

T_and_MIS_per_alpha_filtered = {key: value for key, value in T_and_MIS_per_alpha.items() if value[4] == 1}

# Second: select alpha (which is the key of this dict) based on the largest avg MIS (3rd entry of the dict):

alpha_selected = max(T_and_MIS_per_alpha_filtered, key=lambda x: T_and_MIS_per_alpha_filtered[x][2])

# Third: select T as the max T of the alpha_selected

T_selected = T_and_MIS_per_alpha_filtered[alpha_selected][1]

print('++++++++++++ FINAL ++++++++++++')

print("gamma   = ", [gamma])
print("gamma_c = ", [gamma_c])
print("alpha   = ", [alpha_selected])
print("beta    = ", [beta])
print("T       = ", [T_selected])

print('++++++++++++++++++++++++++++++')

++++++++++++ FINAL ++++++++++++
gamma   =  [149]
gamma_c =  [1]
alpha   =  [0.05]
beta    =  [0.8]
T       =  [np.int64(73)]
++++++++++++++++++++++++++++++
